# ⭐ radix sort를 구현해보자.

## 고정된 3자리 정수 라면

In [33]:
arr = [329, 457, 657, 839, 436, 720, 355]

### **1의 자리 정렬**
#### > 주의 사항
 - bucket을 만들때, 자릿수별로 만들어야 하니 미리 0~9 까지 만들어 놓고, 해당 자리수에 넣어야 하기 때문에 이중리스트로
 - 이중리스트를 풀어서 원소만 넣을것이기 때문에 그대로 추가하는 append 대신 extend로 풀어서 넣기

In [34]:
bucket = [[] for _ in range(10)]

for x in arr:
  digit = x%10
  bucket[digit].append(x)

arr = []

for b in bucket:
  arr.extend(b)

print(arr)

[720, 355, 436, 457, 657, 329, 839]


### 10의 자리 정렬

In [35]:
bucket = [[] for _ in range(10)]

for x in arr:
  digit = (x//10)%10
  bucket[digit].append(x)

arr = []
for b in bucket:
  arr.extend(b)

print(arr)

[720, 329, 436, 839, 355, 457, 657]


### 100의 자리 정렬

In [36]:
bucket = [[] for _ in range(10)]

for x in arr:
  digit = (x//100)%10
  bucket[digit].append(x)

arr = []

for b in bucket:
  arr.extend(b)

print(arr)

[329, 355, 436, 457, 657, 720, 839]


## 가변길이 정수라면

In [37]:
arr = [38295, 291, 692602, 293, 291, 12, 0, 12985]

In [38]:
max_len = len(str(max(arr)))

In [39]:
for i in range(max_len):
  bucket = [[] for _ in range(10)]
  div = 10**i
  for x in arr:
    digit = (x//div) %10
    bucket[digit].append(x)

  arr = []

  for b in bucket:
    arr.extend(b)

In [40]:
print(arr)

[0, 12, 291, 291, 293, 12985, 38295, 692602]


## ❤ Radix Sort 구현하며 겪은 핵심 실수 정리

---

## 1. bucket 구조 오해

❌ 잘못된 생각
`bucket = []`
임시 저장소 하나만 있으면 된다고 생각함.

⭕ 올바른 구조
`bucket = [[] for _ in range(10)]`
→ 0~9 자릿수별 분류함 10개가 필요함.

---


## 2. append 와 extend 혼동
❌ 실수
`arr.append(b)`
→ 리스트 안에 리스트가 들어가 구조 붕괴.

⭕ 정답
`arr.extend(b)`
→ 버킷 안의 값들을 풀어서 삽입.

---

## 3. bucket 초기화 위치 오류
❌ 실수
`bucket = [[] for _ in range(10)]   # for 밖`
→ 이전 단계 데이터가 계속 누적되어 값 폭증.

⭕ 정답
```python
for exp in range(max_len):
    bucket = [[] for _ in range(10)]   # 반드시 for 안
```

---

## 4. 1의 자리 정렬 생략
❌ 실수
`for i in range(1, max_len):`
→ 10의 자리부터 시작 → 안정 정렬 깨짐 → 중복 순서 붕괴.

⭕ 정답
`for exp in range(max_len):`

---


## 5. 내림차순 구현 착각
❌ 실수
`for b in reversed(bucket):`
→ LSD 방식에서 안정성 파괴.

⭕ 정답
오름차순으로 완성 후
`arr.reverse()`

---


## 6. arr = bucket 대입 실수
❌ 실수
`arr = bucket`
→ 2차원 구조로 변해 다음 단계에서 에러.

⭕ 정답
```python
arr = []
for b in bucket:
    arr.extend(b)
```

# ⭐ Floyd-Warshall 을 구현해보자.

## ▶ **Floyd-Warshall 알고리즘** ?

모든 정점 쌍 사이의 최단 거리를 구하는 알고리즘

> 언제써?
> - "A에서 B까지 최단 거리”
> - "B에서 C까지 최단 거리”
> - 이런 걸 모든 쌍에 대해 전부 알고 싶을 때 사용한다.


## ▶ 이차원 리스트의 의미

`dist[i][j]` : `i`번 정점에서 `j`번 정점까지 가는 **현재까지 알고 있는 최단 거리**

> 초기 상태는
> - 자기 자신 -> `0`
> - 간언이 있으면 -> 그 가중치
> - 없으면 -> 매우 큰 값($\infty$)

## ▶ 핵심 아이디어

> `i -> j`로 가는것보다 `i -> k -> j`로 우회하는게 더 짧으면 바꾼다.

이걸 **모든 `i` 모든 `j` 모든 `k`** 조합에 대해 반복

In [41]:
INF = int(1e9)
n = 3
m = 2

In [42]:
# graph 없을때 n, m 입력받기용
# graph = [[INF]*(n+1) for _ in range(n+1)]

# 자기 자신으로 가는 경우를 0 으로 초기화
# for a in range(len(graph)):
#   for b in range(len(graph)):
#     if a == b:
#       graph[a][b] = 0

# 초기 데이터
graph = [
    [0,   3,  float('inf'), 7],
    [8,   0,  2,            float('inf')],
    [5,   float('inf'), 0,  1],
    [2,   float('inf'), float('inf'), 0]
]

In [43]:
%%time
for i in range(len(graph)):
  for j in range(len(graph)):
    for k in range(len(graph)):
      graph[i][j] = min(graph[i][j], graph[i][k] + graph[k][j])

CPU times: user 57 µs, sys: 0 ns, total: 57 µs
Wall time: 60.8 µs


In [44]:
graph

[[0, 3, 5, 6], [7, 0, 2, 3], [3, 6, 0, 1], [2, 5, 7, 0]]

## ❤ 피드백

> k번째 반복이 끝났을 때 `graph[i][j]` 는 중간 노드로 `0 ~ k`까지만 사용하는 단거리 여야 한다.

이 조건을 만족하려면
k가 가장 바깥 루프에 있어야 한다.

In [45]:
n = len(graph)

In [46]:
%%time
for k in range(n):          # 거쳐가는 노드
    for i in range(n):      # 출발 노드
        for j in range(n):  # 도착 노드
            if graph[i][j] > graph[i][k] + graph[k][j]:
                graph[i][j] = graph[i][k] + graph[k][j]


CPU times: user 38 µs, sys: 5 µs, total: 43 µs
Wall time: 45.8 µs


In [47]:
graph

[[0, 3, 5, 6], [5, 0, 2, 3], [3, 6, 0, 1], [2, 5, 7, 0]]